In [53]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request
import lxml

In [54]:
# Extracting 2020 movies from wikepedia
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]

In [55]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [56]:
tables = soup.find_all('table',class_='wikitable sortable')

In [57]:
len(tables)

4

In [58]:
type(tables[0])

bs4.element.Tag

In [59]:
dfs = [df1, df2, df3, df4]
df = pd.concat(dfs, ignore_index=True)

In [60]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,JANUARY,3,The Grudge,Screen Gems / Stage 6 Films / Ghost House Pict...,Nicolas Pesce (director/screenplay); Andrea Ri...,[2],NaN
1,JANUARY,10,Underwater,20th Century Fox / TSG Entertainment / Chernin...,"William Eubank (director); Brian Duffield, Ada...",[3],NaN
2,JANUARY,10,Like a Boss,Paramount Pictures / Artists First,"Miguel Arteta (director); Sam Pitman, Adam Col...",[4],NaN
3,JANUARY,10,Three Christs,IFC Films,Jon Avnet (director/screenplay); Eric Nazarian...,NaN,NaN
4,JANUARY,10,Inherit the Viper,Lionsgate / Barry Films / Tycor International ...,Anthony Jerjen (director); Andrew Crabtree (sc...,[5],NaN
...,...,...,...,...,...,...,...
270,DECEMBER,25,We Can Be Heroes,Netflix / Troublemaker Studios / Double R Prod...,Robert Rodriguez (director/screenplay); Priyan...,NaN,[243]
271,DECEMBER,25,News of the World,Universal Pictures / Perfect World Pictures,Paul Greengrass (director/screenplay); Luke Da...,NaN,[244]
272,DECEMBER,25,One Night in Miami...,Amazon Studios,Regina King (director); Kemp Powers (screenpla...,NaN,[245]
273,DECEMBER,25,Promising Young Woman,Focus Features / FilmNation Entertainment,Emerald Fennell (director/screenplay); Carey M...,NaN,[246]


In [61]:
df.dtypes

Opening                                                                        object
Opening.1                                                                       int64
Title                                                                          object
Production company                                                             object
Cast and crew                                                                  object
.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.    object
Ref.                                                                           object
dtype: object

In [62]:
df_2020 = df[['Title','Cast and crew']]

In [63]:
df_2020

,Title,Cast and crew
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...
1,Underwater,"William Eubank (director); Brian Duffield, Ada..."
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col..."
3,Three Christs,Jon Avnet (director/screenplay); Eric Nazarian...
4,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...
...,...,...
270,We Can Be Heroes,Robert Rodriguez (director/screenplay); Priyan...
271,News of the World,Paul Greengrass (director/screenplay); Luke Da...
272,One Night in Miami...,Regina King (director); Kemp Powers (screenpla...
273,Promising Young Woman,Emerald Fennell (director/screenplay); Carey M...


In [64]:
# pip install tmdbv3api

In [65]:
from tmdbv3api import TMDb, Movie
import requests
import numpy as np

tmdb = TMDb()
tmdb.api_key = '4a1cec1f5a88caebd4b3390d00560d8e'
tmdb_movie = Movie()

def get_genre(x):
    try:
        result = tmdb_movie.search(x)
        # print("Result:", result)  # Debug statement
        if result:
            movie_id = result[0].id
            # print("Movie ID:", movie_id)  # Debug statement
            response = requests.get(f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb.api_key}')
            data_json = response.json()
            # print("Data JSON:", data_json)  # Debug statement
            if 'genres' in data_json:
                genres = [genre['name'] for genre in data_json['genres']]
                return ' '.join(genres)
        return np.NaN
    except Exception as e:
        print(f"An error occurred: {e}")
        return np.NaN


In [66]:
df_2020['genres'] = df_2020['Title'].apply(get_genre) 

An error occurred: getattr(): attribute name must be string
An error occurred: getattr(): attribute name must be string


C:\Users\hp\AppData\Local\Temp\ipykernel_6652\3896718950.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['genres'] = df_2020['Title'].apply(get_genre)


In [67]:
df_2020

,Title,Cast and crew,genres
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Horror Mystery Thriller
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",Horror Science Fiction Action Adventure
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Comedy
3,Three Christs,Jon Avnet (director/screenplay); Eric Nazarian...,Drama
4,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Crime Thriller Drama
...,...,...,...
270,We Can Be Heroes,Robert Rodriguez (director/screenplay); Priyan...,Family Action Fantasy Comedy
271,News of the World,Paul Greengrass (director/screenplay); Luke Da...,Drama Western Adventure
272,One Night in Miami...,Regina King (director); Kemp Powers (screenpla...,Drama
273,Promising Young Woman,Emerald Fennell (director/screenplay); Carey M...,Thriller Crime Drama


In [68]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [69]:
df_2020['director_name'] = df_2020['Cast and crew'].map(lambda x: get_director(str(x)))

C:\Users\hp\AppData\Local\Temp\ipykernel_6652\3848435694.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['director_name'] = df_2020['Cast and crew'].map(lambda x: get_director(str(x)))


In [70]:

def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [71]:
df_2020['actor_1_name'] = df_2020['Cast and crew'].map(lambda x: get_actor1(str(x)))

C:\Users\hp\AppData\Local\Temp\ipykernel_6652\3646981363.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['actor_1_name'] = df_2020['Cast and crew'].map(lambda x: get_actor1(str(x)))


In [72]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [73]:
df_2020['actor_2_name'] = df_2020['Cast and crew'].map(lambda x: get_actor2(str(x)))

C:\Users\hp\AppData\Local\Temp\ipykernel_6652\463054939.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['actor_2_name'] = df_2020['Cast and crew'].map(lambda x: get_actor2(str(x)))


In [74]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [75]:
df_2020['actor_3_name'] = df_2020['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [76]:
df_2020

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Horror Mystery Thriller,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",Horror Science Fiction Action Adventure,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Comedy,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek
3,Three Christs,Jon Avnet (director/screenplay); Eric Nazarian...,Drama,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins
4,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Crime Thriller Drama,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs
...,...,...,...,...,...,...,...
270,We Can Be Heroes,Robert Rodriguez (director/screenplay); Priyan...,Family Action Fantasy Comedy,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin
271,News of the World,Paul Greengrass (director/screenplay); Luke Da...,Drama Western Adventure,Paul Greengrass,Tom Hanks,Helena Zengel,NaN
272,One Night in Miami...,Regina King (director); Kemp Powers (screenpla...,Drama,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge
273,Promising Young Woman,Emerald Fennell (director/screenplay); Carey M...,Thriller Crime Drama,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie


In [77]:
df_2020 = df_2020.rename(columns={'Title':'movie_title'})

In [78]:
new_df20 = df_2020.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [79]:
new_df20

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Horror Mystery Thriller,The Grudge
1,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Horror Science Fiction Action Adventure,Underwater
2,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Comedy,Like a Boss
3,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins,Drama,Three Christs
4,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Crime Thriller Drama,Inherit the Viper
...,...,...,...,...,...,...
270,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin,Family Action Fantasy Comedy,We Can Be Heroes
271,Paul Greengrass,Tom Hanks,Helena Zengel,NaN,Drama Western Adventure,News of the World
272,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge,Drama,One Night in Miami...
273,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie,Thriller Crime Drama,Promising Young Woman


In [80]:
new_df20['comb'] = new_df20['actor_1_name'] + ' ' + new_df20['actor_2_name'] + ' '+ new_df20['actor_3_name'] + ' '+ new_df20['director_name'] +' ' + new_df20['genres']

In [81]:
new_df20.isna().sum()

director_name     0
actor_1_name      0
actor_2_name      5
actor_3_name     28
genres            2
movie_title       0
comb             29
dtype: int64

In [82]:
new_df20 = new_df20.dropna(how='any')

In [83]:
new_df20.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [84]:
new_df20['movie_title'] = new_df20['movie_title'].str.lower()

C:\Users\hp\AppData\Local\Temp\ipykernel_6652\2267385682.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df20['movie_title'] = new_df20['movie_title'].str.lower()


In [85]:
new_df20['comb'] =  new_df20['movie_title'] + ' ' + new_df20['comb'] 

C:\Users\hp\AppData\Local\Temp\ipykernel_6652\1274073338.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df20['comb'] =  new_df20['movie_title'] + ' ' + new_df20['comb']


In [86]:
new_df20['comb'][0]

'the grudge Andrea Riseborough Demián Bichir John Cho Nicolas Pesce Horror Mystery Thriller'

In [87]:
old_df = pd.read_csv('../datasets/final_data.csv')
old_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,avatar CCH Pounder Joel David Moore Wes Studi ...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,pirates of the caribbean: at world's end Johnn...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,spectre Christoph Waltz Rory Kinnear Stephanie...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,the dark knight rises Tom Hardy Christian Bale...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,star wars: episode vii - the force awakens ...
...,...,...,...,...,...,...,...
6091,Pete Docter,Jamie Foxx,Tina Fey,Graham Norton,Animation Family Comedy Fantasy,soul,soul Jamie Foxx Tina Fey Graham Norton Pete Do...
6092,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin,Family Action Fantasy Comedy,we can be heroes,we can be heroes Priyanka Chopra Jonas Pedro P...
6093,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge,Drama,one night in miami...,one night in miami... Kingsley Ben-Adir Eli Go...
6094,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie,Thriller Crime Drama,promising young woman,promising young woman Carey Mulligan Bo Burnha...


In [88]:
 final_df = old_df.append(new_df20,ignore_index=True)

C:\Users\hp\AppData\Local\Temp\ipykernel_6652\2472109381.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = old_df.append(new_df20,ignore_index=True)


In [89]:
final_df.sample(10)

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
5399,Ava DuVernay,Storm Reid,Oprah Winfrey,Reese Witherspoon,Adventure Science Fiction Family Fantasy,a wrinkle in time,a wrinkle in time Storm Reid Oprah Winfrey Ree...
6052,Edoardo Ponti,Sophia Loren,Ibrahima Gueye,Abril Zamora,Drama,the life ahead,the life ahead Sophia Loren Ibrahima Gueye Abr...
4072,James Dodson,Larry Miller,Anupam Kher,Shriya Saran,Comedy Drama Romance,the other end of the line,the other end of the line Larry Miller Anupam ...
376,Peter Berg,Dwayne Johnson,Rosario Dawson,Ewen Bremner,Action Adventure Comedy Thriller,the rundown,the rundown Dwayne Johnson Rosario Dawson Ewen...
6306,Joel Crawford,Nicolas Cage,Emma Stone,Ryan Reynolds,Animation Family Adventure Fantasy Comedy,the croods: a new age,the croods: a new age Nicolas Cage Emma Stone ...
2610,Katja von Garnier,Olivier Martinez,Agnes Bruckner,Chris Geere,Drama Fantasy Horror Romance,blood and chocolate,blood and chocolate Olivier Martinez Agnes Bru...
2117,Kevin Reynolds,Peter Firth,Jan Cornet,María Botto,Action Adventure Drama Mystery,risen,risen Peter Firth Jan Cornet María Botto Kevin...
4874,Andrew Bujalski,Andrew Bujalski,Kate Dollenmayer,Justin Rice,Comedy Drama,funny ha ha,funny ha ha Andrew Bujalski Kate Dollenmayer J...
3160,Tom Holland,Catherine Hicks,Alex Vincent,Dinah Manoff,Fantasy Horror,child's play,child's play Catherine Hicks Alex Vincent Dina...
25,Peter Berg,Liam Neeson,Alexander Skarsgård,Tadanobu Asano,Action Adventure Sci-Fi Thriller,battleship,battleship Liam Neeson Alexander Skarsgård Tad...


In [90]:
final_df.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [91]:
final_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,avatar CCH Pounder Joel David Moore Wes Studi ...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,pirates of the caribbean: at world's end Johnn...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,spectre Christoph Waltz Rory Kinnear Stephanie...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,the dark knight rises Tom Hardy Christian Bale...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,star wars: episode vii - the force awakens ...
...,...,...,...,...,...,...,...
6337,Pete Docter,Jamie Foxx,Tina Fey,Graham Norton,Animation Family Comedy Fantasy,soul,soul Jamie Foxx Tina Fey Graham Norton Pete Do...
6338,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin,Family Action Fantasy Comedy,we can be heroes,we can be heroes Priyanka Chopra Jonas Pedro P...
6339,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge,Drama,one night in miami...,one night in miami... Kingsley Ben-Adir Eli Go...
6340,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie,Thriller Crime Drama,promising young woman,promising young woman Carey Mulligan Bo Burnha...


In [92]:
final_df.to_csv('../datasets/final_data.csv',index=False)